In [10]:
import numpy as np

In [75]:
class TicTacToeGame:
    
    def __init__(self):
        self.squares = np.array([" "," "," "," "," "," "," "," "," "])
        
    def reset(self):
        self.squares = np.array([" "," "," "," "," "," "," "," "," "])
        
    def availableSquares(self):
        return np.where(self.squares == " ")[0]
    
    def x_move(self,idx):
        if idx > 9:
            print("Illegal move X at " + str(idx))
        elif self.squares[idx] != " ":
            print("Illegal move X at " + str(idx) + ".  Space occupied.")
            return
        else:
            self.squares[idx] = "X"
            
        return self.squares.copy()

    def o_move(self,idx):
        if idx > 8:
            print("Illegal move O at " + str(idx))
        elif self.squares[idx] != " ":
            print("Illegal move O at " + str(idx) + ".  Space occupied.")
            return
        else:
            self.squares[idx] = "O"
            
        return self.squares.copy()
    
    def showBoard(self):
        s = self.squares
        print("")
        print(" " + s[0] + " | " + s[1] + " | " + s[2])
        print("-----------")
        print(" " + s[3] + " | " + s[4] + " | " + s[5])
        print("-----------")
        print(" " + s[6] + " | " + s[7] + " | " + s[8])

    def showMiniBoard(self):
        self.renderMiniboard(self.squares)
    
    def renderMiniboard(self,arr):
        s = arr
        print("")
        temp = ""
        for i in range(9):
            if s[i] == " ":
                temp += ". "
            else:
                temp += s[i] + " "
            if (i+1) % 3 == 0:
                print(temp)
                temp = ""
    
    def evaluateForWin(self,indices):
        winningTuples = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
        for win in winningTuples:
            if len(np.where(np.in1d(win, indices))[0]) == 3:
                return True
        return False
            
    def didXWin(self):
        x_squares = np.where(self.squares == "X")[0]
        return self.evaluateForWin(x_squares)

    def didOWin(self):
        o_squares = np.where(self.squares == "O")[0]
        return self.evaluateForWin(o_squares)

    def renderMiniGame(self,steps):
        lines = ["","",""]
        for s in steps:
            line_idx = 0
            for i in range(9):
                if s[i] == " ":
                    lines[line_idx] += ". "
                else:
                    lines[line_idx] += s[i] + " "
                if (i+1) % 3 == 0:
                    lines[line_idx] += "  "
                    line_idx += 1
        for l in lines:
            print(l)
                    

In [ ]:
value_table = {}

In [78]:
def evaluateBoard(arr):
    key = tuple(arr)
    if key in value_table:
        return value_table[key]
    else:
        value_table[key] = 0.0
        return 0.0

def setValue(squares,val):
    key = tuple(squares)
    value_table[key] = val
    
def evaluateOptions(game):
    available = game.availableSquares()
    
    lines = ["","","",""]

    current = game.squares
    
    game.showMiniBoard()
    print("")
    
    for idx in available:
        s = current.copy()
        s[idx] = "X"
        line_idx = 0
        for i in range(9):
            if s[i] == " ":
                lines[line_idx] += ". "
            else:
                lines[line_idx] += s[i] + " "
            if (i+1) % 3 == 0:
                lines[line_idx] += "  "
                line_idx += 1
        lines[3] +=  '{:5.2f}'.format(evaluateBoard(s)) + "   "
    for l in lines:
        print(l)
    print("")
    
def chooseBestOption(game):
    available = game.availableSquares()
    current = game.squares
    
    choice = available[0]
    value = 0.0
    
    for idx in available:
        s = current.copy()
        s[idx] = "X"
        newVal = evaluateBoard(s)
        if newVal > value:
            choice = idx
            value = newVal
    
    return choice


def updateValues(currentState,nextState):
    cKey = tuple(currentState)
    nKey = tuple(nextState)
    
    if nKey in value_table:
        nextVal = value_table[nKey]
    else:
        value_table[nKey] = 0.0
        nextVal = 0.0
    
    if cKey in value_table:
        currentVal = value_table[cKey]
        newVal = currentVal + (0.9*(nextVal - currentVal))
        value_table[cKey] = newVal
    else:
        value_table[cKey] = 0.9 * nextVal
    

In [86]:
for i in range(10):
    game = TicTacToeGame()
    steps = []
    game_end = False
    while game_end == False:

        evaluateOptions(game)
        available = game.availableSquares()
        action = available[np.random.randint(len(available))]
#         action = chooseBestOption(game)
        
        currentState = game.squares.copy()
        steps.append(game.x_move(action))
        nextState = game.squares.copy()
#         game.showMiniBoard()
        
        if game.didXWin():
            setValue(nextState,1)
            updateValues(currentState,nextState)
            game_end = True
            print("")
            print("WIN")
            print("")
            break

        updateValues(currentState,nextState)
        available = game.availableSquares()
        
        if len(available) == 0:
            game_end = True
            print("")
            print("NO WINNER")
            print("")
            break
            
        currentState = game.squares.copy()
        opponentAction = available[np.random.randint(len(available))]
        steps.append(game.o_move(opponentAction))
        nextState = game.squares.copy()
#         game.showMiniBoard()

        if game.didOWin():
            setValue(game.squares,-1)
            updateValues(currentState,nextState)
            game_end = True
            print("")
            print("LOSS")
            print("")
            break

        updateValues(currentState,nextState)

    game.renderMiniGame(steps)
    print("")



. . . 
. . . 
. . . 

X . .   . X .   . . X   . . .   . . .   . . .   . . .   . . .   . . .   
. . .   . . .   . . .   X . .   . X .   . . X   . . .   . . .   . . .   
. . .   . . .   . . .   . . .   . . .   . . .   X . .   . X .   . . X   
-0.37   -0.76    0.78   -0.29    0.79    0.23   -0.68    0.86    0.93   


. . . 
. . O 
. . X 

X . .   . X .   . . X   . . .   . . .   . . .   . . .   
. . O   . . O   . . O   X . O   . X O   . . O   . . O   
. . X   . . X   . . X   . . X   . . X   X . X   . X X   
 0.82    0.08    0.67    0.53    0.72    0.27    0.17   


X . . 
. . O 
O . X 

X X .   X . X   X . .   X . .   X . .   
. . O   . . O   X . O   . X O   . . O   
O . X   O . X   O . X   O . X   O X X   
-0.49    0.99    0.09    1.00    0.82   


WIN

. . .   . . .   X . .   X . .   X . .   
. . .   . . O   . . O   . . O   . X O   
. . X   . . X   . . X   O . X   O . X   


. . . 
. . . 
. . . 

X . .   . X .   . . X   . . .   . . .   . . .   . . .   . . .   . . .   
. . .   . . .   . 

In [81]:
for key, val in value_table.items():
    print(str(key) + " -> " + str(val))

(' ', 'X', ' ', ' ', ' ', ' ', ' ', ' ', ' ') -> 0.27457714847504217
(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ') -> 0.9854222146939048
(' ', 'X', ' ', ' ', ' ', ' ', ' ', ' ', 'O') -> 0.0398036622068878
(' ', 'X', 'X', ' ', ' ', ' ', ' ', ' ', 'O') -> 0.8803881473938129
(' ', 'X', 'X', ' ', ' ', ' ', ' ', 'O', 'O') -> 0.8512202042774326
(' ', 'X', 'X', ' ', 'X', ' ', ' ', 'O', 'O') -> -0.8840324070000002
(' ', 'X', 'X', 'O', 'X', ' ', ' ', 'O', 'O') -> 0.9810989999918992
('X', 'X', 'X', 'O', 'X', ' ', ' ', 'O', 'O') -> 1
('X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ') -> 0.7076980236040638
('X', ' ', ' ', ' ', ' ', ' ', 'O', ' ', ' ') -> 0.25595242503877
('X', 'X', ' ', ' ', ' ', ' ', 'O', ' ', ' ') -> 0.6177930267989924
('X', 'X', ' ', 'O', ' ', ' ', 'O', ' ', ' ') -> -0.3529095801177873
('X', 'X', ' ', 'O', ' ', ' ', 'O', ' ', 'X') -> 0.9851889062438093
('X', 'X', 'O', 'O', ' ', ' ', 'O', ' ', 'X') -> -0.6574271686927202
('X', 'X', 'O', 'O', ' ', ' ', 'O', 'X', 'X') -> 0.801801981800

('O', 'X', 'O', 'X', 'X', ' ', 'O', ' ', ' ') -> 0.9999899937089992
('O', 'X', 'O', 'X', 'X', ' ', 'O', ' ', 'X') -> 0.9999998972009999
('O', 'X', 'O', 'X', 'X', 'O', 'O', ' ', 'X') -> 0.9999999999999999
('X', 'O', 'O', ' ', ' ', ' ', ' ', ' ', 'X') -> 0.99439629974073
('X', 'O', 'O', 'X', ' ', ' ', ' ', ' ', 'X') -> 0.9998915156781301
('X', 'O', 'O', 'X', 'O', ' ', ' ', ' ', 'X') -> -0.8180000206920083
('X', 'O', 'O', 'X', 'O', 'X', ' ', ' ', 'X') -> -0.99999999999
('X', 'O', 'O', 'X', 'O', 'X', ' ', 'O', 'X') -> -1
('O', ' ', ' ', ' ', ' ', ' ', 'X', ' ', 'X') -> -0.34859210722513445
('O', ' ', ' ', 'O', ' ', ' ', 'X', ' ', 'X') -> 0.9780138683234023
('O', ' ', ' ', 'O', 'X', ' ', 'X', ' ', 'X') -> 0.9674425438339727
('O', ' ', 'O', 'O', 'X', ' ', 'X', ' ', 'X') -> 0.9964988189149337
('O', 'X', 'O', 'O', 'X', ' ', 'X', ' ', 'X') -> 0.009099009899997193
('O', 'X', 'O', 'O', 'X', 'O', 'X', ' ', 'X') -> 1.0
('O', 'X', 'O', 'O', 'X', 'O', 'X', 'X', 'X') -> 1
(' ', ' ', 'O', 'X', ' ', 'X'

('O', 'O', 'X', ' ', ' ', 'X', ' ', ' ', 'X') -> 1
(' ', ' ', ' ', 'X', ' ', ' ', 'X', ' ', 'O') -> 0.6732613913140453
(' ', ' ', ' ', 'X', 'O', ' ', 'X', ' ', 'O') -> -0.9382096592082
(' ', ' ', ' ', 'X', 'O', 'X', 'X', ' ', 'O') -> -0.8206395299999999
(' ', 'O', 'X', 'X', 'O', 'X', 'X', 'O', 'O') -> -1
(' ', ' ', 'X', ' ', 'O', 'O', ' ', 'X', 'X') -> 0.998023095
(' ', ' ', 'X', ' ', 'O', 'O', 'O', 'X', 'X') -> -0.834462
('X', ' ', 'X', ' ', 'O', 'O', 'O', 'X', 'X') -> -0.09090900000000002
('O', ' ', ' ', 'X', 'O', ' ', 'X', ' ', 'X') -> -0.9572175
('O', ' ', ' ', 'X', 'O', 'O', 'X', ' ', 'X') -> 0.9171072891090001
('O', 'X', ' ', 'X', 'O', 'O', 'X', ' ', 'X') -> 0.9908981099999999
('O', 'X', ' ', 'X', 'O', 'O', 'X', 'O', 'X') -> 0.0
('O', 'X', 'X', 'X', ' ', 'X', 'O', ' ', 'O') -> -0.9999998999999999
('O', 'X', 'X', 'X', 'O', 'X', 'O', ' ', 'O') -> -1
(' ', ' ', 'X', 'O', 'X', ' ', ' ', ' ', ' ') -> 0.8302314293249282
('O', 'X', 'X', ' ', ' ', 'O', 'X', 'X', 'O') -> -0.80180180109027

('O', ' ', 'X', 'O', ' ', ' ', ' ', 'X', 'X') -> 0.997930431
('O', 'O', 'X', 'O', ' ', ' ', ' ', 'X', 'X') -> 0.9983688488999999
(' ', 'X', 'O', ' ', 'O', 'X', ' ', ' ', ' ') -> 0.01631868620926679
(' ', 'X', 'O', ' ', 'O', 'X', ' ', ' ', 'X') -> -0.003064172872329015
(' ', ' ', 'X', 'O', 'X', 'X', 'O', 'X', 'O') -> -0.90990099
(' ', 'X', 'X', ' ', ' ', 'O', ' ', ' ', 'O') -> -0.85030225805262
(' ', 'X', 'X', ' ', ' ', 'O', 'X', ' ', 'O') -> 0.09516624470999996
(' ', 'X', 'X', 'O', ' ', 'O', 'X', ' ', 'O') -> 0.99
(' ', 'X', 'X', 'O', ' ', 'O', 'X', 'X', 'O') -> -0.99819900009
('X', 'O', 'X', 'O', 'X', 'X', ' ', ' ', 'O') -> 0.09009890009999999
(' ', 'O', ' ', ' ', ' ', 'O', 'X', ' ', 'X') -> 0.9816776600946481
(' ', 'O', ' ', ' ', 'X', 'O', 'X', ' ', 'X') -> 0.9807245027181001
('O', 'O', ' ', ' ', 'X', 'O', 'X', ' ', 'X') -> 0.6756542605079999
('X', 'X', ' ', ' ', ' ', 'O', 'X', 'O', 'O') -> 0.9999835378281892
(' ', 'X', ' ', 'O', 'X', 'X', 'O', ' ', 'O') -> -0.9836802
('O', ' ', 'X',

('X', 'X', ' ', 'O', ' ', 'O', ' ', 'X', 'O') -> -0.8019989972100001
('X', 'O', ' ', 'X', 'X', 'O', ' ', 'O', ' ') -> 0.9
('X', ' ', ' ', 'O', 'O', 'X', 'X', ' ', ' ') -> -0.0712922925601458
('X', 'X', ' ', 'O', 'O', 'X', ' ', ' ', 'O') -> 0.18826037999757006
('O', ' ', 'X', 'X', 'O', 'X', ' ', 'O', 'X') -> 1
('X', ' ', 'O', 'X', 'O', 'O', ' ', 'X', ' ') -> -0.8982900080999999
('X', 'O', 'X', 'X', 'O', 'O', ' ', 'X', ' ') -> 0.89999999991
('X', 'O', 'O', ' ', 'X', ' ', 'O', 'X', ' ') -> 0.998748
('X', ' ', 'O', 'O', 'O', 'X', 'X', ' ', ' ') -> 0.008908379999999997
(' ', ' ', 'O', 'O', 'O', 'X', 'X', ' ', 'X') -> 0.098099075699983
(' ', 'X', ' ', 'X', 'X', 'O', 'O', 'O', ' ') -> 0.09720720979946192
('X', 'X', ' ', 'O', 'X', 'O', ' ', 'O', 'X') -> 1
('O', ' ', ' ', 'X', 'X', 'O', 'X', 'O', 'X') -> 0.09089999990099995
('X', 'O', 'X', 'O', ' ', ' ', 'X', ' ', 'O') -> 0.02603860163225999
(' ', 'O', 'X', ' ', 'O', ' ', 'O', 'X', 'X') -> 0.9890999999991008
(' ', 'X', 'O', 'O', ' ', 'X', 'O', 